# Collaborative Filtering

Predictions for user rankings 
Dataset: https://grouplens.org/datasets/movielens/

## Preliminaries

### Imports

In [1]:
import os
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split,KFold

%matplotlib inline

### Random Seed

In [2]:
seed=5543
np.random.seed(seed)

### Data

In [3]:
raw_dir="../raw/ml-1m"
data_dir="../data/MovieLens"

In [4]:
if not (os.path.exists(data_dir)):
    os.mkdir(data_dir)

In [5]:
filename=raw_dir+"/ratings.dat"

In [6]:
data_all=pd.read_csv(filename,sep="::",header=None,names=["userId","movieId","rating","TimeStamp"])

/home/manel/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [7]:
data_all.head()

,userId,movieId,rating,TimeStamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


### Train Test Split

In this example we do not handle carefuly new users or new movies (we just asign them at random).

A production product would treat new users/movies separately.

In [8]:
userEncoder=LabelEncoder()
movieEncoder=LabelEncoder()

In [9]:
users_all=userEncoder.fit_transform(data_all[["userId"]].values.ravel())
movies_all=movieEncoder.fit_transform(data_all[["movieId"]].values.ravel())
ratings_all=data_all[["rating"]].values.ravel()

In [10]:
users,users_test,movies,movies_test,ratings, ratings_test=train_test_split(users_all,movies_all,ratings_all,test_size=0.15)
users_train,users_val, movies_train,movies_val, ratings_train,ratings_val=train_test_split(users,movies,ratings,test_size=0.15)
users_train.shape,users_val.shape,users_test.shape

((722650,), (127527,), (150032,))

In [11]:
unkown_users=~np.isin(users_val,users)
unknown_movies=~np.isin(movies_val,movies)
print(unkown_users.sum(),unknown_movies.sum())

0 0


In [12]:
unique_users=userEncoder.classes_
unique_movies=movieEncoder.classes_

In [13]:
N_users=len(unique_users)
N_movies=len(unique_movies)
print(N_users,N_movies)

6040 3706


## Collaborative Filter

### Mean Rating

Mean rating over the training set is 
	\begin{equation}
			\mu = \frac{1}{N_\mathcal{T}}\sum_{(i,u)\in\mathcal{T}} r_{u,i}
		\end{equation}
And, we define the differential rating

\begin{equation}
    \Delta r_{u,i} = r_{u,i} - \mu
\end{equation}

In [14]:
mu=ratings_train.mean()
drating=np.mean((ratings_train-mu)**2)
print(mu,drating)

3.5812398809935653 1.2487443009293984


### Parameter Initialization

We implement the following initialization
\begin{align}
	b_u^0  &\sim \mathcal{N}(0, 10^{-4}) \\
	b_i^0  &\sim \mathcal{N}(0, 10^{-4}) \\
	p_{u,f}^0  &\sim \mathcal{N}\left(0, \frac{1}{\max({1},\sqrt{F})}\right) \\
	q_{i,f}^0  &\sim \mathcal{N}\left(0, \frac{1}{\max({1},\sqrt{F})}\right) \\
\end{align}

In [15]:
def initialize_params(F,N_users,N_movies):
    b_users=np.random.normal(0,0.0001,N_users)
    b_movies=np.random.normal(0,0.0001,N_movies)
    p_users=np.random.normal(0,1/max(1,np.sqrt(F)),(N_users,F))
    p_movies=np.random.normal(0,1//max(1,np.sqrt(F)),(N_movies,F))
    return b_users,b_movies,p_users,p_movies

In [17]:
F=2

In [18]:
b_users,b_movies,p_users,p_movies=initialize_params(F,N_users,N_movies)
params=[mu,b_users,b_movies,p_users,p_movies]

### Rating Model

The model prediction for $r$ is given by
\begin{equation}
	\hat{r}_{i,u} =\mu + b_u + b_i + p_u^T q_i
\end{equation}

In [19]:
def predict_rating(users,movies,params):
    mu,b_users,b_movies,p_users,p_movies=params 
    b_u=b_users[users]
    b_m=b_movies[movies]
    p_u=p_users[users]
    p_m=p_movies[movies]
    prod=np.sum(p_u*p_m,axis=1)
    r_hat=mu+b_u+b_m+prod
    return r_hat

In [20]:
R=predict_rating(users_train,movies_train,params)
R[:10]

array([3.5811178 , 3.58137423, 3.58115786, 3.58146872, 3.58122582,
       3.58112864, 3.58093923, 3.58123367, 3.58094814, 3.58119053])

### Loss Function

The lost function is 
\begin{equation}
	L(\theta;\{r\}) = \frac{1}{N_\mathcal{S}} \sum_{{u,i}\in \mathcal{S}} \left( r_{u,i} - \hat{r}_{u,i}\right)^2
\end{equation}

In [21]:
def rating_error(users,movies,rating,params0):
    dr=rating-predict_rating(users,movies,params0)
    return np.mean(dr**2)

In [22]:
rating_error(users_train,movies_train,ratings_train,params)

1.248742966734941

### Learning Step

We will implement a step of stochastic gradient descent as
\begin{align}
	b_u & \leftarrow  b_u +  \gamma \Delta r_{u,i} \\
	b_i &\leftarrow b_i + \gamma \Delta r_{u,i} \\
	p_u & \leftarrow p_u + \gamma\left(q_{i}\Delta r_{u,i} - \lambda p_u\right) \\
	q_i &\leftarrow q_i + \gamma \left(p_{u}\Delta r_{u,i} - \lambda q_u \right)  
	\label{eq:step}
\end{align}


In [23]:
def learning_step(user,movie,rating,parms0,penalty,batch_size):
    N=len(rating)
    mu,b_users,b_movies,p_users,p_movies=parms0
    perm=np.random.permutation(len(rating))
    for i1 in range(0,N,batch_size):
        idx=perm[i1:i1+batch_size]       
        u=user[idx]
        m=movie[idx]
        r=rating[idx]
        b_u=b_users[u]
        b_m=b_movies[m]
        p_u=p_users[u]
        p_m=p_movies[m]
        prod=np.sum(p_u*p_m,axis=1)
        r_hat=mu+b_u+b_m+prod
        dr=r-r_hat
        b_users[u] +=learning_rate*(dr) 
        b_movies[m]+=learning_rate*(dr) 
        p_users[u] +=learning_rate*(dr[:,np.newaxis]*p_m-penalty*p_u) 
        p_movies[m]+=learning_rate*(dr[:,np.newaxis]*p_u-penalty*p_m) 
    return 

### Training Function

Given the hyperparameters we just train for a fixed number of epochs


In [24]:
def fit_ratings(users_train,movies_train,ratings_train,users_val,movies_val,ratings_val,
                F,learning_rate,penalty,steps,batch_size):
    
    mu=ratings_train.mean()
    b_users,b_movies,p_users,p_movies=initialize_params(F,N_users,N_movies)
    parms=[mu,b_users,b_movies,p_users,p_movies]
    for i1 in range(steps):  
        loss=rating_error(users_train,movies_train,ratings_train,parms)    
        learning_step(users_train,movies_train,ratings_train,parms,penalty,batch_size)
        if i1 % (steps//10)==0:
            val_loss=rating_error(users_val,movies_val,ratings_val,parms)
            print("\t",i1,loss,val_loss)
    loss=rating_error(users_train,movies_train,ratings_train,parms)
    val_loss=rating_error(users_val,movies_val,ratings_val,parms)
    print("\tFinal",loss,val_loss)
    return val_loss,parms

### Model Hyper-parameters

In [25]:
learning_rate=0.005
penalty=0.1
batch_size=50
steps=200

### Train Popularity Model

In [26]:
loss,params=fit_ratings(users_train,movies_train,ratings_train,
            users_val,  movies_val, ratings_val,
           0,learning_rate,penalty,10,batch_size
           )

	 0 1.248746642725307 0.8909740526539852
	 1 0.8789293209282619 0.8551414073649105
	 2 0.8413121945710145 0.8422269447426097
	 3 0.827094512576697 0.8356778554860488
	 4 0.8199053667259332 0.831931600744101
	 5 0.8155451055453977 0.8298184162152527
	 6 0.8130133859324009 0.8284141083235405
	 7 0.8111815395205997 0.8274597323768608
	 8 0.8099879740889979 0.8269773267236272
	 9 0.8090191069252736 0.8264105330218547
	Final 0.8081858378145389 0.8264105330218547


### Train Model with interaction Term

Here we asume the embedding space has dimension $F=2$.

In [ ]:
loss,params=fit_ratings(users_train,movies_train,ratings_train,
            users_val,  movies_val, ratings_val,
           F,learning_rate,penalty,steps,batch_size,
           )

	 0 1.2487413272977004 0.8919647278968952
	 20 0.8005053824001939 0.825574098155686
	 40 0.788783145543379 0.8150789791241544
	 60 0.758732024001011 0.7911134361845616
	 80 0.7444282606397932 0.781016275974047
	 100 0.737458723836546 0.7765548494867625
	 120 0.7336375008227668 0.7746195277755271
	 140 0.7312443829111536 0.7729298978347575
	 160 0.7299435936606753 0.7722617994295322
	 180 0.7288682180958788 0.7721030687230466


### Colaborative Filter Model

We grap the hyperparameters, training and prediction in a single model for ease of use.

In [32]:
class Recommender:
    def __init__(self,F,penalty,learning_rate,steps,batch_size):
        self.F=F
        self.penalty=penalty
        self.learning_rate=learning_rate
        self.steps=steps
        self.batch_size=batch_size
    def fit(self,users,movies,ratings, users_val,movies_val,ratings_val):
        loss,params=fit_ratings(users,movies,ratings,
                                users_val,movies_val,ratings_val,
                                self.F,self.learning_rate,self.penalty,self.steps,self.batch_size
                               )
        self.params=params
        return loss
    def predict(users,movies):
        return predict_ratings(users,movies,self.params)
        

In [33]:
model=Recommender(F=5,penalty=0.1,learning_rate=0.05,steps=10,batch_size=50)
model.fit(users_train,movies_train,ratings_train,users_val,movies_val,ratings_val)

	 0 1.2487491435047675 0.890884965767691
	 1 0.87400637899907 0.856136779916064
	 2 0.8350615442590369 0.8431626271230641
	 3 0.8203097886918276 0.8367159809528412
	 4 0.8126704877072 0.8329638983922121
	 5 0.8082523436197229 0.8307115543495848
	 6 0.8053462488336441 0.8291730605342861
	 7 0.8032047189870908 0.8278535652366308
	 8 0.8017586224206004 0.8271560419529516
	 9 0.8006674468903492 0.8265487010644992
	Final 0.7997206726910681 0.8265487010644992


0.8265487010644992

## Parameter Search

We do a grid search using a single validation set to find the range of penalties and embedding dimension that seems to perform best

In [34]:
results=[]
best_loss=1e10
best_F=None
best_penalty=None

if True:
  for F in [1,5,10,20,30,50,100,150]: #[0,1,5,10,20,30,50,75,100,125,150]:
    for penalty in [0,0.01,0.05,0.1,0.15,0.2,1]:
        print()
        print(f"F {F}, penalty {penalty} :")
        model=Recommender(F,penalty,learning_rate,steps,batch_size)
        loss=model.fit(users_train,movies_train,ratings_train,
                               users_val,movies_val,ratings_val)
        results.append((F,penalty,loss))
        if loss<best_loss:
            best_loss=loss
            best_F=F
            best_penalty=penalty
        print()
        print(f"==> {F},{penalty},{loss} == best ({best_F},{best_penalty},{best_loss}) =============")


F 1, penalty 0 :
	 0 2.2297649479240245 0.9591224199407644
	 20 0.800027721408294 0.8317069799262912
	 40 0.776291919894767 0.8106944190050412
	 60 0.7555207747682228 0.7926947065372381
	 80 0.7469745213146038 0.7863286922835332
	 100 0.7430728667900206 0.7839665374894451
	 120 0.7411201514829009 0.7831638351470969
	 140 0.7400930048694438 0.7828983985853032
	 160 0.7395459726436394 0.7827856116910031
	 180 0.7392379293105136 0.7825441717557904
	Final 0.7389403532602861 0.7826964523321334

==> 1,0,0.7826964523321334 == best (1,0,0.7826964523321334) =============

F 1, penalty 0.01 :
	 0 2.2485388893105505 0.9581940121057958
	 20 0.7997113100616658 0.8307259255153069
	 40 0.7747837595677601 0.8100976893921636
	 60 0.7543478579093974 0.7921530045357693
	 80 0.7459384269882637 0.7854553987794991
	 100 0.7425184607850186 0.783650513294766
	 120 0.7408652896808117 0.7822694239538839
	 140 0.7399139852292305 0.7821140811268591
	 160 0.7394934117556343 0.7825503539970973
	 180 0.739226342822

	 80 0.5678322612806114 0.7880309905321662
	 100 0.557953596979457 0.7920196461534071
	 120 0.5517125395907355 0.79657186339414
	 140 0.5476926159708011 0.8011968864532487
	 160 0.5447004981543985 0.8052805572848525
	 180 0.5422889775109342 0.8094709525557573
	Final 0.5406301359755135 0.8130560846720991

==> 10,0,0.8130560846720991 == best (5,0.05,0.7413946506514998) =============

F 10, penalty 0.01 :
	 0 1.2487408423102824 0.8907739173228147
	 20 0.7029405081481221 0.8001532327775172
	 40 0.6188467937531164 0.7765614941920417
	 60 0.5862814339541935 0.7739454484155872
	 80 0.5702575698116711 0.7755615706152033
	 100 0.5610566570673127 0.778418043204533
	 120 0.5551455234811868 0.7805271846391059
	 140 0.5510285430296783 0.782134131019194
	 160 0.5480546333758612 0.7846154863500892
	 180 0.5458037218894956 0.7863915095668054
	Final 0.5439875140230992 0.787761535071537

==> 10,0.01,0.787761535071537 == best (5,0.05,0.7413946506514998) =============

F 10, penalty 0.05 :
	 0 1.248740107

	 120 0.3537032208016508 1.0025413102219385
	 140 0.3471894608570364 1.025512089003799
	 160 0.3422890417820217 1.0463367157344274
	 180 0.3383783546066635 1.0658894703985182
	Final 0.33528237154528345 1.0832290304915235

==> 30,0,1.0832290304915235 == best (10,0.05,0.7379060900644814) =============

F 30, penalty 0.01 :
	 0 1.2487428510543386 0.8904681066673268
	 20 0.5847297432272531 0.7935230809208715
	 40 0.45487988826260467 0.8216609834940338
	 60 0.40873657313014394 0.8491557242462137
	 80 0.38610229762643267 0.870232220782774
	 100 0.37264834806959934 0.8865629799540112
	 120 0.3636555466016144 0.9000983857361838
	 140 0.3572439803751318 0.9115758789385032
	 160 0.35243369740026004 0.9207825701785373
	 180 0.34865430332673036 0.928797768457507
	Final 0.3454936528775615 0.935764351444119

==> 30,0.01,0.935764351444119 == best (10,0.05,0.7379060900644814) =============

F 30, penalty 0.05 :
	 0 1.2487395606555187 0.8905031507624955
	 20 0.6933827999988861 0.7759309833636263
	 40 0

	 160 0.066931851925765 1.4964246097115375
	 180 0.06329813175922755 1.5420511665195873
	Final 0.060390747665515075 1.5821005456321011

==> 100,0,1.5821005456321011 == best (10,0.05,0.7379060900644814) =============

F 100, penalty 0.01 :
	 0 1.2487482241055308 0.8904365925412794
	 20 0.43152955731674686 0.784711105298026
	 40 0.21622261310371252 0.8772954489434904
	 60 0.1543893997963666 0.9358607201096283
	 80 0.1280616540132085 0.9722551594712924
	 100 0.1138245069702643 0.9975231591879773
	 120 0.10489431552353777 1.0154248778036723
	 140 0.09880415438816571 1.0292241231927635
	 160 0.09433823970683937 1.0395513730799868
	 180 0.09092255476158692 1.0475232781469013
	Final 0.08821537940745296 1.0537538407067202

==> 100,0.01,1.0537538407067202 == best (10,0.05,0.7379060900644814) =============

F 100, penalty 0.05 :
	 0 1.2487416344140596 0.89016477847669
	 20 0.6617552900744972 0.7643882355714287
	 40 0.4868316953528183 0.7324146131670651
	 60 0.3917248191207323 0.7345031971503837


In [35]:
fits=pd.DataFrame(results,columns=["F","penalty","val_rms"])
fits.head()

,F,penalty,val_rms
0,1,0.00,0.782696
1,1,0.01,0.782128
2,1,0.05,0.781258
3,1,0.10,0.787065
4,1,0.15,0.798355


In [36]:
summary=pd.pivot_table(fits,index="F",columns="penalty",values="val_rms")
summary

penalty,0.0,0.01,0.05,0.1,0.15,0.2,1.0
F,,,,,,,
1,0.782696,0.782128,0.781258,0.787065,0.798355,0.815372,0.825193
5,0.764203,0.758633,0.741395,0.754068,0.788218,0.813963,0.825245
10,0.813056,0.787762,0.737906,0.747374,0.786918,0.813867,0.825278
20,0.952808,0.870673,0.749341,0.744736,0.787138,0.813556,0.825290
30,1.083229,0.935764,0.754461,0.742779,0.786446,0.813682,0.825612
50,1.302623,1.032195,0.754754,0.741674,0.786159,0.813525,0.825358
100,1.582101,1.053754,0.748517,0.741486,0.786485,0.813546,0.825746
150,1.522999,0.979739,0.738657,0.741149,0.786583,0.813672,0.825255


In [37]:
best=fits.iloc[fits["val_rms"].idxmin()]
best

F          10.000000
penalty     0.050000
val_rms     0.737906
Name: 16, dtype: float64

In [38]:
import seaborn as sns

cm = sns.light_palette("#60FF60", reverse=True, as_cmap=True)

s = summary.style.highlight_min(axis=None)
s

penalty,0.0,0.01,0.05,0.1,0.15,0.2,1.0
F,,,,,,,
1,0.782696,0.782128,0.781258,0.787065,0.798355,0.815372,0.825193
5,0.764203,0.758633,0.741395,0.754068,0.788218,0.813963,0.825245
10,0.813056,0.787762,0.737906,0.747374,0.786918,0.813867,0.825278
20,0.952808,0.870673,0.749341,0.744736,0.787138,0.813556,0.82529
30,1.08323,0.935764,0.754461,0.742779,0.786446,0.813682,0.825612
50,1.30262,1.0322,0.754754,0.741674,0.786159,0.813525,0.825358
100,1.5821,1.05375,0.748517,0.741486,0.786485,0.813546,0.825746
150,1.523,0.979739,0.738657,0.741149,0.786583,0.813672,0.825255


In [39]:
F_best=int(best["F"])
penalty_best=best["penalty"]
F_best,penalty_best

(10, 0.05)

## Cross Validation

It seems $F\approx 10$ with penalty around 0.05 gives best results.

We use 5-Fold cross validation to find the optimal value of $F$

In [40]:
K=5
kfold=KFold(K,shuffle=True)
folds=[]
for fold in kfold.split(users):
    folds.append(fold)

In [41]:
def ratings_cross_validate(model,users,movies,ratings,folds):
    accuracies=[]
    count=0
    for train,val in folds:
        print()
        print("============= Fold",count+1,"===========")
        users_train=users[train]
        movies_train=movies[train]
        ratings_train=ratings[train]
        users_val=users[val]
        movies_val=movies[val]
        ratings_val=ratings[val]
        loss=model.fit(users_train,movies_train,ratings_train,
                                   users_val,movies_val,ratings_val)
        
        accuracies.append(loss)
        print("======= fold",count+1,"loss",loss,"============")
        print()
        count+=1
    accuracies=np.array(accuracies)
    return accuracies.mean()

In [58]:
results=[]
best_loss=1e10
best_F=None
steps=100
if True:
  for F in [5,10,15]: 
        print()
        print(f"F {F} :")
        model=Recommender(F,best_penalty,learning_rate,steps,batch_size)
        loss=ratings_cross_validate(model,users,movies,ratings,
                               folds)
        results.append((F,loss))
        if loss<best_loss:
            best_loss=loss
            best_F=F
        print()
        print(f"==> {F},{penalty},{loss} == best ({best_F},{best_penalty},{best_loss}) =============")


F 5 :

============= Fold 1 ===========
	 0 1.2485533530336144 0.8932260820557525
	 10 0.792153645151391 0.8284441461561335
	 20 0.7596851350083867 0.8073201016757684
	 30 0.7291479115139694 0.7905257533022896
	 40 0.7057719194647919 0.7788425942450067
	 50 0.6892993059143258 0.7706883615736058
	 60 0.6777306704150264 0.7652558488252055
	 70 0.6692703522347676 0.7611975269798796
	 80 0.6627616923298618 0.7585839984881941
	 90 0.6580722714403028 0.7560365320775337
	Final 0.6544426100756603 0.754407832790789
======= fold 1 loss 0.754407832790789 ============


============= Fold 2 ===========
	 0 1.2478524832474827 0.8962957102144979
	 10 0.7922599406899052 0.8333267782960897
	 20 0.7670963705646401 0.8174016801086219
	 30 0.7343180194957236 0.7978551599139871
	 40 0.7100195764880174 0.7845717707513513
	 50 0.6914439061011414 0.7748161832518049
	 60 0.6778810060704996 0.7676659654676805
	 70 0.6686360374904579 0.763427531750924
	 80 0.6619130533879918 0.760341507843067
	 90 0.6572000510

	 50 0.6009716181301853 0.7580625474067376
	 60 0.582379256415953 0.7558669642638263
	 70 0.5692704228510136 0.7543656458231661
	 80 0.5596501254639026 0.7537558141946534
	 90 0.5524028808630612 0.7537394345606137
	Final 0.5466941424695451 0.7537372173462676
======= fold 5 loss 0.7537372173462676 ============


==> 15,1,0.7519847069644519 == best (10,0.05,0.7495350542252549) =============


## Test Best model

Seems best model is really $F=10$ with penalty $0.02$, so we test performance on the test set

In [59]:
model=Recommender(best_F,penalty_best,learning_rate,steps,batch_size)
loss=model.fit(users,movies,ratings,users_test,movies_test,ratings_test)
print(best_F,penalty_best,loss)

	 0 1.2487151147154587 0.8734996712312967
	 10 0.7854459353697039 0.8168954476243606
	 20 0.7298827082827767 0.7789219089035535
	 30 0.6857647008788721 0.7546617468115456
	 40 0.6565707493132016 0.7414681641613389
	 50 0.6377613938778485 0.7344088473281033
	 60 0.6254117611629383 0.7306500346148762
	 70 0.6170301391922364 0.7286495844998203
	 80 0.611048728955314 0.7274774636837298
	 90 0.606653455554716 0.7263877950958051
	Final 0.6032863549775624 0.7261302778562422
10 0.05 0.7261302778562422


In [60]:
loss

0.7261302778562422

We have achieved a $\approx 0.73$ mean square error, a 12% improvement in accuracy over the 0.83 mean square error of the popularity model. 